### This notebook will be for processing and cleaning the subway turnstile dataset for 2019
- Will need to determine all the turnstile differences for entries between current and last value
- Same for exits 
- This can give me a representation of how many people enter/exit station X in a given time period
- Do this for only stations in Manhattan

In [3]:
import json
import pandas as pd

### Explanation of linking the 3 datasets to get useful information:
- turnstile_data is the dataset obtained from: https://data.ny.gov/Transportation/Turnstile-Usage-Data-2019/xfn5-qji9
- This dataset contains all the info in 2019 for all the turnstiles in the MTA system
- station_data is a dataset obtained from: http://web.mta.info/developers/data/nyct/subway/Stations.csv
- This dataset contains information for all the stations in the MTA system
- The problem that occurs is how to link the two datasets together
- lookup_data provides a link between the two in the form of a column named remote or unit id that links each remote unit id to each station

In [4]:
turnstile_data = pd.read_csv('Turnstile_Usage_Data__2019.csv')
turnstile_data.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-00-05,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,17538854,7031168
1,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
2,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
3,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389


In [5]:
with open('stations.csv') as file:
    station_data = pd.read_csv(file)
station_data.head()

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
0,1,1.0,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2.0,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3.0,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4.0,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5.0,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


In [6]:
with open('remote_complex_lookup.csv') as file:
    lookup_data = pd.read_csv(file)
lookup_data.head()

,remote,booth,complex_id,station,line_name,division
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


In [7]:
#count the number of null values for complex id
lookup_data['complex_id'].isnull().sum()

36

In [8]:
#change the complex id column to object instead of float
lookup_data['complex_id'] = lookup_data['complex_id'].astype(str)
lookup_data.head()

,remote,booth,complex_id,station,line_name,division
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


### First lets look at all the station_data dataset
- First I will drop all rows that are not in Manhattan

In [9]:
for i in range(len(station_data)):
    if station_data['borough'][i] == 'Q' or  station_data['borough'][i] == 'Bk' or station_data['borough'][i] == 'Bx'or station_data['borough'][i] == 'SI':
        station_data.drop(i, inplace=True)

station_data.head()

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
6,7,7.0,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn
7,8,8.0,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn
8,9,9.0,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn
9,10,10.0,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn
10,11,11.0,611,R16,BMT,Broadway - Brighton,Times Sq - 42 St,M,N Q R W,Subway,40.754672,-73.986754,Uptown & Queens,Downtown & Brooklyn


In [10]:
station_data.tail(20)

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
472,473,475.0,475,Q05,IND,Second Av,96 St,M,Q,Subway,40.784318,-73.947152,NaN,Downtown & Brooklyn
473,474,476.0,476,Q04,IND,Second Av,86 St,M,Q,Subway,40.777891,-73.951787,Uptown,Downtown & Brooklyn
474,475,477.0,477,Q03,IND,Second Av,72 St,M,Q,Subway,40.768799,-73.958424,Uptown,Downtown & Brooklyn
496,497,NaN,R468,NaN,RIT,NaN,Roosevelt Island Tram - Eastbound,NaN,NaN,NaN,40.761186,-73.964191,NaN,NaN
497,498,NaN,R469,NaN,RIT,NaN,Roosevelt Island Tram - Westbound,NaN,NaN,NaN,40.757307,-73.954097,NaN,NaN
498,499,NaN,R540,NaN,PTH,NaN,World Trade Center,NaN,NaN,NaN,40.712052,-74.014129,NaN,NaN
499,500,NaN,R541,NaN,PTH,NaN,33 St,NaN,NaN,NaN,40.747850,-73.989875,NaN,NaN
500,501,NaN,R542,NaN,PTH,NaN,23 St,NaN,NaN,NaN,40.742672,-73.994177,NaN,NaN
501,502,NaN,R543,NaN,PTH,NaN,Exchange Pl,NaN,NaN,NaN,40.716554,-74.033372,NaN,NaN
502,503,NaN,R544,NaN,PTH,NaN,Harrison,NaN,NaN,NaN,40.740472,-74.157344,NaN,NaN


In [11]:
#take out the last 17 station names and put them in a list
station_names = station_data['stop name'].tail(17).tolist()
station_names

['Roosevelt Island Tram - Eastbound',
 'Roosevelt Island Tram - Westbound',
 'World Trade Center',
 '33 St',
 '23 St',
 'Exchange Pl',
 'Harrison',
 '14 St',
 'Pavonia/Newport',
 '9 St',
 'Christopher St',
 'Newark Penn Station',
 'Hoboken',
 'Grove St',
 'Journal Sq',
 'Howard Beach',
 'Jamaica']

In [12]:
station_names_to_drop=['Roosevelt Island Tram - Eastbound',
 'Roosevelt Island Tram - Westbound',
 'Exchange Pl',
 'Harrison',
 'Pavonia/Newport',
 '9 St',
 'Newark Penn Station',
 'Hoboken',
 'Grove St',
 'Journal Sq',
 'Howard Beach',
 'Jamaica']

 # Create a boolean mask to identify rows with names are in the list
mask = station_data['stop name'].isin(station_names_to_drop)

# Apply the mask to drop the rows
station_data = station_data[~mask]

station_data.tail(10)
    

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
470,471,469.0,610,901,IRT,Lexington - Shuttle,Grand Central - 42 St,M,S,Subway,40.752769,-73.979189,Times Sq,NaN
471,472,471.0,471,726,IRT,Flushing,34 St - 11 Av,M,7,Subway,40.755882,-74.001910,Queens,NaN
472,473,475.0,475,Q05,IND,Second Av,96 St,M,Q,Subway,40.784318,-73.947152,NaN,Downtown & Brooklyn
473,474,476.0,476,Q04,IND,Second Av,86 St,M,Q,Subway,40.777891,-73.951787,Uptown,Downtown & Brooklyn
474,475,477.0,477,Q03,IND,Second Av,72 St,M,Q,Subway,40.768799,-73.958424,Uptown,Downtown & Brooklyn
498,499,NaN,R540,NaN,PTH,NaN,World Trade Center,NaN,NaN,NaN,40.712052,-74.014129,NaN,NaN
499,500,NaN,R541,NaN,PTH,NaN,33 St,NaN,NaN,NaN,40.747850,-73.989875,NaN,NaN
500,501,NaN,R542,NaN,PTH,NaN,23 St,NaN,NaN,NaN,40.742672,-73.994177,NaN,NaN
503,504,NaN,R545,NaN,PTH,NaN,14 St,NaN,NaN,NaN,40.738507,-73.998741,NaN,NaN
506,507,NaN,R548,NaN,PTH,NaN,Christopher St,NaN,NaN,NaN,40.732593,-74.008924,NaN,NaN


In [13]:
#set the last 5 rows column[gtfs stop id] to the complex id value
station_data.loc[station_data.tail(5).index, 'gtfs stop id'] = station_data['complex id'].tail(5)

In [14]:
#change complex id in station to have .0 and be of type string
station_data['complex id'] = station_data['complex id'].astype(str)
station_data['complex id'] = station_data['complex id'] + '.0'

### Now match up complex_id in station_data with complex_id in lookup_data

In [15]:
station_remote_info = pd.merge(station_data, lookup_data, left_on='complex id', right_on='complex_id')
station_remote_info.head()

,ogc_fid,station id,complex id,gtfs stop id,division_x,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,remote,booth,complex_id,station,line_name,division_y
0,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R016,N305A,613.0,LEXINGTON-53 ST,EM6,IND
1,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R017,N306,613.0,LEXINGTON-53 ST,EM6,IND
2,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R017,N305,613.0,LEXINGTON-53 ST,EM6,IND
3,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R050,R244,613.0,59 ST,456NQR,IRT
4,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R050,R244A,613.0,59 ST,456NQR,IRT


In [16]:
#drop the following columns: ogc_fid, gtfs stop id, division_x, line, daytime routes, structure, north direction label, south direction label, line_name, division_y
station_remote_info.drop(['ogc_fid', 'station id', 'complex id','gtfs stop id', 'division_x', 'line', 'borough','daytime routes', 'structure', 'north direction label', 'south direction label', 'line_name', 'division_y'], axis=1, inplace=True)

station_remote_info.head()

,stop name,gtfs latitude,gtfs longitude,remote,booth,complex_id,station
0,Lexington Av/59 St,40.76266,-73.967258,R016,N305A,613.0,LEXINGTON-53 ST
1,Lexington Av/59 St,40.76266,-73.967258,R017,N306,613.0,LEXINGTON-53 ST
2,Lexington Av/59 St,40.76266,-73.967258,R017,N305,613.0,LEXINGTON-53 ST
3,Lexington Av/59 St,40.76266,-73.967258,R050,R244,613.0,59 ST
4,Lexington Av/59 St,40.76266,-73.967258,R050,R244A,613.0,59 ST


In [17]:
#rename the columns:
station_remote_info.rename(columns={'remote ':'remote_unit_id','gtfs longitude':'lon','gtfs latitude': 'lat', 'stop name': 'station_name', 'station':'short_name'}, inplace=True)
station_remote_info.head()


,station_name,lat,lon,remote_unit_id,booth,complex_id,short_name
0,Lexington Av/59 St,40.76266,-73.967258,R016,N305A,613.0,LEXINGTON-53 ST
1,Lexington Av/59 St,40.76266,-73.967258,R017,N306,613.0,LEXINGTON-53 ST
2,Lexington Av/59 St,40.76266,-73.967258,R017,N305,613.0,LEXINGTON-53 ST
3,Lexington Av/59 St,40.76266,-73.967258,R050,R244,613.0,59 ST
4,Lexington Av/59 St,40.76266,-73.967258,R050,R244A,613.0,59 ST


In [18]:
len(station_remote_info)

527

### Lets now look at the turnstile_data dataset
- First I should drop all the rows that concern Manhattan stations only

In [19]:
len(turnstile_data)

10467101

In [20]:
#reduce the number of rows in turnstile data by only including matches for the following condition: 
#remote in remote=unit. This is a unique id for each subsection of a station
turnstile_data = turnstile_data[turnstile_data['Unit'].isin(station_remote_info['remote_unit_id'])]
len(turnstile_data)

4617524

### This should now be all the turnstile data for manhattan stations in 2019

- Use the remote_unit_id for linking a station to a turnstile
- station_remote_info is useful as it has the station name, lat and lon and also a link to the turnstile data for that station
- Can now proceed with processing the turnstile data

- I will convert date and time to unix timestamp, name: timestamp
- I will combine C/A, Unit, SCP and the timestamp to create a unique identifier for each row, name: id
- I will combine C/A, Unit and SCP to create a unique id for each, name: turnstile_id
- I will calculate the entry and exit values for each row, name: entry_diff and exit_diff

In [41]:
#create a new dateframe name for turnstile data
turnstile_data_man = turnstile_data.copy()

In [42]:
turnstile_data_man.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-00-05,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,17538854,7031168
1,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
2,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
3,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389


In [43]:
#convert date and time to unix timestamp
turnstile_data_man['DateTime'] = pd.to_datetime(turnstile_data_man['Date'] + ' ' + turnstile_data_man['Time'])

turnstile_data_man['Timestamp'] = turnstile_data_man['DateTime'].apply(lambda x: x.timestamp())

In [44]:
#combine c/a, unit, and scp to create a unique id for each turnstile
turnstile_data_man['turnstile_id'] = turnstile_data_man['C/A'] +' '+ turnstile_data_man['Unit'] +' '+  turnstile_data_man['SCP']

In [45]:
#combine c/a, unit, scp, and timestamp to create a unique id for each turnstile at a given time
turnstile_data_man['id'] = turnstile_data_man['turnstile_id']+ ' '+  turnstile_data_man['Timestamp'].astype(str)

In [46]:
#can now drop c/a, scp, line name, divison,date, time, datetime
turnstile_data_man.drop(['C/A', 'SCP', 'Line Name', 'Division', 'Date', 'Time', 'DateTime'], axis=1, inplace=True)

In [47]:
turnstile_data_man.columns

Index(['Unit', 'Station', 'Description', 'Entries',
       'Exits                                                     ',
       'Timestamp', 'turnstile_id', 'id'],
      dtype='object')

In [48]:
#re-orient the columns
#1st will be id, 2nd turnstile id, then unit, timestamp, station, entires,exits and description
new_order = ['id', 'turnstile_id', 'Unit', 'Timestamp', 'Station', 'Entries', 'Exits                                                     ', 'Description']
turnstile_data_man = turnstile_data_man[new_order]


In [49]:
#re-name the columns:
turnstile_data_man.rename(columns={'Unit':'remote_unit_id', 'Timestamp':'time_stamp','Station':'station_name', 'Entries':'entries', 'Exits                                                     ':'exits', 'Description':'description'}, inplace=True)
len(turnstile_data_man)

4617524

In [50]:
#check for duplicate ids, Nan and other possible error values
#count duplicates:
# is it possible that entry and exit values differ for the duplicate id?
#create a new column
turnstile_data_man['check_dup']=turnstile_data_man['id']+' '+turnstile_data_man['entries'].astype(str)+' '+turnstile_data_man['exits'].astype(str)

duplicate_count = turnstile_data_man['id'].duplicated().sum()
print("Number of duplicates in 'id' column:", duplicate_count)
dup_count = turnstile_data_man['check_dup'].duplicated().sum()
print("Number of duplicates in check_dup column:", dup_count)
nan_count=turnstile_data_man.isnull().sum()
print("Number of null values in all columns:\n", nan_count)
# count the number of times "RECOVR AUD" appears in the description column
r_aud=turnstile_data_man['description'].str.contains('RECOVR AUD').sum()
print("Number of times 'RECOVR AUD' appears in the description column:", r_aud)


Number of duplicates in 'id' column: 181348
Number of duplicates in check_dup column: 181211
Number of null values in all columns:
 id                0
turnstile_id      0
remote_unit_id    0
time_stamp        0
station_name      0
entries           0
exits             0
description       0
check_dup         0
dtype: int64
Number of times 'RECOVR AUD' appears in the description column: 21902


In [51]:
#drop column check_dup
turnstile_data_man.drop(['check_dup'], axis=1, inplace=True)
#drop all the duplicate id values as only 137 of them have diff entry and exit values
turnstile_data_man = turnstile_data_man.drop_duplicates(subset=['id'])

In [52]:
#sort by time stamp and name of station, drop all data before 1546300800
turnstile_data_man.sort_values(by=['time_stamp', 'station_name'], inplace=True)

#drop all data before 1546300800
turnstile_data_man = turnstile_data_man[turnstile_data_man['time_stamp'] >= 1546290000]

#### Left to do:
- Calculate entry and exit differences for each turnstile
- Drop the description column

In [53]:
len(turnstile_data_man)


4399477

In [54]:
turnstile_data_man.head(50)

,id,turnstile_id,remote_unit_id,time_stamp,station_name,entries,exits,description
10406998,R138 R293 00-03-03 1546290000.0,R138 R293 00-03-03,R293,1.546290e+09,34 ST-PENN STA,158353,233074,REGULAR
10406999,R138 R293 00-02-05 1546290000.0,R138 R293 00-02-05,R293,1.546290e+09,34 ST-PENN STA,53313,59850,REGULAR
10407000,R138 R293 00-02-03 1546290000.0,R138 R293 00-02-03,R293,1.546290e+09,34 ST-PENN STA,1569424,974507,REGULAR
10407001,R138 R293 00-00-02 1546290000.0,R138 R293 00-00-02,R293,1.546290e+09,34 ST-PENN STA,22123940,11841883,REGULAR
10407002,R138 R293 00-03-01 1546290000.0,R138 R293 00-03-01,R293,1.546290e+09,34 ST-PENN STA,5076351,6276942,REGULAR
10407003,R138 R293 00-06-00 1546290000.0,R138 R293 00-06-00,R293,1.546290e+09,34 ST-PENN STA,6895211,5302078,REGULAR
10407004,R138 R293 00-02-01 1546290000.0,R138 R293 00-02-01,R293,1.546290e+09,34 ST-PENN STA,5889422,2033752,REGULAR
10407005,R138 R293 00-03-00 1546290000.0,R138 R293 00-03-00,R293,1.546290e+09,34 ST-PENN STA,98405984,1155428577,REGULAR
10407006,R138 R293 00-03-04 1546290000.0,R138 R293 00-03-04,R293,1.546290e+09,34 ST-PENN STA,156894,139918,REGULAR
10407007,R138 R293 00-00-03 1546290000.0,R138 R293 00-00-03,R293,1.546290e+09,34 ST-PENN STA,1273222,699949,REGULAR
